# Capstone Part03
*Exploring Visualisation Techniques for Aviation Accident Analysis*
> NOTE: The maps from folium and plotly are not very jupyternotebook-github friendly. The maps have been saved in the project folder to view in Github.
- ./world_map_accidents.html
- ./world_heatmap_accidents.png
PS: its best when viewed on jupyter notebook as it is interactive and the text labels are better when hovered.

## Submitted by Roshan Lulu

# Introduction

### 1. Plot Crash Locations on the world map

In [1]:
import folium
import pandas as pd

In [2]:
# Write save it to a csv file
data = pd.read_csv('./dataset/02_crashes_geolocation.csv')

In [3]:
data.head()

,SubLocation,Country,latlon,place,sub_latlon,sub_place
0,unknown,Peru,"(-9.189967, -75.015152)",Peru,"(41.6262707, -79.6735841)","Unknown, 101 W Spring St, Titusville, PA 16354..."
1,at an unknown location,Germany,"(51.165691, 10.451526)",Germany,NaN,NaN
2,"Boca Chica, Tamaulipas",Mexico,"(23.634501, -102.552784)",Mexico,"(25.9516829, -97.3341488)","Boca Chica Blvd, Brownsville, TX, USA"
3,Akyab Main RAF StationAKY),Myanmar,"(21.916221, 95.955974)",Myanmar (Burma),NaN,NaN
4,near Río Putumayo,Brazil,"(-14.235004, -51.92528)",Brazil,"(-2.6584061, -69.7378901)",Putumayo River


In [4]:
data['Country'].isnull().sum()
data[data['Country'].isnull()]
data.dropna(subset=['Country'], inplace=True)

In [5]:
data['Country'].isnull().sum()

0

In [6]:
data['latlon'].isnull().sum()
data[data['latlon'].isnull()]

,SubLocation,Country,latlon,place,sub_latlon,sub_place


In [7]:
# Split Country lat and long to different columns
data['latlon'] = [latlong.replace('(','').replace(')','') for latlong in data['latlon']]

In [8]:
data['latitude'] = [latlong.split(',')[0] for latlong in data['latlon']]
data['longitude'] = [latlong.split(',')[1] for latlong in data['latlon']]

## I have to manipulate the Dataframe to Plot.. 
*There are around 8712 datapoints. If I try to plot each point, my system is not able to handle it. Since I am plotting Country wise, for now I can manage to plot countries just once. When it comes to actual location plotting I will have to either divide them up into different locaions or make it possible somehow. Only time will tell... *

In [9]:
data_con = data[['latlon','Country']]

In [10]:
data_tem = pd.DataFrame(data_con['Country'].value_counts())
data_tem.reset_index(inplace = True)
# data_tem['index'].value_counts()

In [11]:
data_con = pd.DataFrame(data_con['latlon'].value_counts())

In [12]:
data_con.reset_index(inplace = True)

In [13]:
data_con['Country'] = data_tem['index']

In [14]:
data_con.columns = ['latlon', 'count','Country']
data_con.head()

,latlon,count,Country
0,"37.09024, -95.712891",1581,United States of America
1,"61.52401, 105.318756",431,Russia
2,"55.378051, -3.435973",280,United Kingdom
3,"56.130366, -106.346771",274,Canada
4,"-14.235004, -51.92528",223,Brazil


In [15]:
data_con['latitude'] = [latlong.split(',')[0] for latlong in data_con['latlon']]
data_con['longitude'] = [latlong.split(',')[1] for latlong in data_con['latlon']]

In [16]:
(data_con.head())

,latlon,count,Country,latitude,longitude
0,"37.09024, -95.712891",1581,United States of America,37.09024,-95.712891
1,"61.52401, 105.318756",431,Russia,61.52401,105.318756
2,"55.378051, -3.435973",280,United Kingdom,55.378051,-3.435973
3,"56.130366, -106.346771",274,Canada,56.130366,-106.346771
4,"-14.235004, -51.92528",223,Brazil,-14.235004,-51.92528


In [18]:
#create a map
this_map = folium.Map(prefer_canvas=True)

def plotDot(point):
    '''input: series that contains a numeric named latitude and a numeric named longitude
    this function creates a CircleMarker and adds it to your this_map'''
#     print(point['latlon'])
    folium.CircleMarker(location=[point.latitude,point.longitude],
                        popup=point['Country'],
                        fill_color='#132b5e',
                        radius=5,
                        weight=0).add_to(this_map)
    
# plotDot(data[0])
# plotDot(data[1])

#use df.apply(,axis=1) to "iterate" through every row in dataframe
data_con.apply(plotDot, axis = 1)
# Set the zoom to the maximum possible
this_map.fit_bounds(this_map.get_bounds())
#Save the map to an HTML file
this_map.save('world_map_accidents.html')
this_map

## 2. Plot Countries by Accident Count in terms of heat map

In [19]:
import branca.colormap as cm

In [20]:
blRd = cm.LinearColormap(['blue', 'red'],
                           vmin=min(data_con['count']), vmax=max(data_con['count']))
blRd

In [21]:
import plotly.plotly as py
import pandas as pd
import pycountry

## Updating country names to suit the pycountry library I have used

In [22]:
data_con.loc[data_con['Country'] == 'United States of America', ['Country']] = 'United States'
data_con.loc[data_con['Country'] == 'Russia', ['Country']] = 'Russian Federation'
data_con.loc[data_con['Country'] == 'Vietnam', ['Country']] = 'Viet Nam'
data_con.loc[data_con['Country'] == 'Bolivia', ['Country']] = 'Bolivia, Plurinational State of'
data_con.loc[data_con['Country'] == 'Democratic Republic)', ['Country']] = 'Congo, The Democratic Republic of the'
data_con.loc[data_con['Country'] == 'Venezuela', ['Country']] = 'Venezuela, Bolivarian Republic of'
data_con.loc[data_con['Country'] == 'Iran', ['Country']] = 'Iran, Islamic Republic of'
data_con.loc[data_con['Country'] == 'Laos', ['Country']] = "Lao People's Democratic Republic"
data_con.loc[data_con['Country'] == 'Taiwan', ['Country']] = 'Taiwan, Province of China'
data_con.loc[data_con['Country'] == 'Pacific Ocean', ['Country']] = 'Fiji'
data_con.loc[data_con['Country'] == 'South Korea', ['Country']] = "Korea, Democratic People's Republic of"
data_con.loc[data_con['Country'] == 'Tanzania', ['Country']] = 'Tanzania, United Republic of'
data_con.loc[data_con['Country'] == 'Czech Republic', ['Country']] = 'Czechia'
data_con.loc[data_con['Country'] == 'Syria', ['Country']] = 'Syrian Arab Republic'
data_con.loc[data_con['Country'] == "Cote d'Ivoire", ['Country']] = "Côte d'Ivoire"
data_con.loc[data_con['Country'] == "Falkland Islands", ['Country']] = 'Falkland Islands (Malvinas)'
data_con.loc[data_con['Country'] == "U.S. Virgin Islands", ['Country']] = 'Virgin Islands, U.S.'
data_con.loc[data_con['Country'] == "British Virgin Islands", ['Country']] = 'Virgin Islands, British'
data_con.loc[data_con['Country'] == "North Korea", ['Country']] = 'Korea, Republic of'
data_con.loc[data_con['Country'] == "St. Pierre and Miquelon", ['Country']] = 'Saint Pierre and Miquelon'
data_con.loc[data_con['Country'] == "Micronesia", ['Country']] = 'Micronesia, Federated States of'
data_con.loc[data_con['Country'] == "Reunion", ['Country']] = 'Réunion'
data_con.loc[data_con['Country'] == "St. Kitts and Nevis", ['Country']] = 'Saint Kitts and Nevis'
data_con.loc[data_con['Country'] == "East Timor", ['Country']] = 'Timor-Leste'
data_con.loc[data_con['Country'] == "Keeling) Islands", ['Country']] = 'Cocos (Keeling) Islands'
data_con.loc[data_con['Country'] == "Moldova", ['Country']] = 'Moldova, Republic of'
data_con.loc[data_con['Country'] == "U.S. Minor Outlying Islands", ['Country']] = 'United States Minor Outlying Islands'




data_con.loc[data_con['Country'] == 'Indian Ocean', ['Country']] = 'Brazil'
data_con.loc[data_con['Country'] == 'North Pole', ['Country']] = 'Brazil'
data_con.loc[data_con['Country'] == 'Mediterranean Sea', ['Country']] = 'Brazil'
data_con.loc[data_con['Country'] == 'Atlantic Ocean', ['Country']] = 'Brazil'
data_con.loc[data_con['Country'] == 'Unknown country', ['Country']] = 'Brazil'
data_con.loc[data_con['Country'] == "FYROM)", ['Country']] = 'Brazil'
data_con.head()

,latlon,count,Country,latitude,longitude
0,"37.09024, -95.712891",1581,United States,37.09024,-95.712891
1,"61.52401, 105.318756",431,Russian Federation,61.52401,105.318756
2,"55.378051, -3.435973",280,United Kingdom,55.378051,-3.435973
3,"56.130366, -106.346771",274,Canada,56.130366,-106.346771
4,"-14.235004, -51.92528",223,Brazil,-14.235004,-51.92528


In [23]:
input_countries = data_con['Country']

countries = {}
for country in pycountry.countries:
    countries[country.name] = country.alpha_3
countries

{'Afghanistan': 'AFG',
 'Albania': 'ALB',
 'Algeria': 'DZA',
 'American Samoa': 'ASM',
 'Andorra': 'AND',
 'Angola': 'AGO',
 'Anguilla': 'AIA',
 'Antarctica': 'ATA',
 'Antigua and Barbuda': 'ATG',
 'Argentina': 'ARG',
 'Armenia': 'ARM',
 'Aruba': 'ABW',
 'Australia': 'AUS',
 'Austria': 'AUT',
 'Azerbaijan': 'AZE',
 'Bahamas': 'BHS',
 'Bahrain': 'BHR',
 'Bangladesh': 'BGD',
 'Barbados': 'BRB',
 'Belarus': 'BLR',
 'Belgium': 'BEL',
 'Belize': 'BLZ',
 'Benin': 'BEN',
 'Bermuda': 'BMU',
 'Bhutan': 'BTN',
 'Bolivia, Plurinational State of': 'BOL',
 'Bonaire, Sint Eustatius and Saba': 'BES',
 'Bosnia and Herzegovina': 'BIH',
 'Botswana': 'BWA',
 'Bouvet Island': 'BVT',
 'Brazil': 'BRA',
 'British Indian Ocean Territory': 'IOT',
 'Brunei Darussalam': 'BRN',
 'Bulgaria': 'BGR',
 'Burkina Faso': 'BFA',
 'Burundi': 'BDI',
 'Cabo Verde': 'CPV',
 'Cambodia': 'KHM',
 'Cameroon': 'CMR',
 'Canada': 'CAN',
 'Cayman Islands': 'CYM',
 'Central African Republic': 'CAF',
 'Chad': 'TCD',
 'Chile': 'CHL',
 

In [24]:
codes = [pycountry.countries.get(name = country).alpha_3 for country in input_countries]

In [25]:
data_con['Code'] = codes
data_con.head()

,latlon,count,Country,latitude,longitude,Code
0,"37.09024, -95.712891",1581,United States,37.09024,-95.712891,USA
1,"61.52401, 105.318756",431,Russian Federation,61.52401,105.318756,RUS
2,"55.378051, -3.435973",280,United Kingdom,55.378051,-3.435973,GBR
3,"56.130366, -106.346771",274,Canada,56.130366,-106.346771,CAN
4,"-14.235004, -51.92528",223,Brazil,-14.235004,-51.92528,BRA


In [31]:
# df = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/2014_world_gdp_with_codes.csv')
scl = [[1.0, 'rgb(84,39,143)'], [0.8, 'rgb(117,107,177)'],[0.6, 'rgb(158,154,200)'],[0.4, 'rgb(188,189,220)'],\
       [0.2, 'rgb(218,218,235)'], [0.0, 'rgb(242,240,247)']]


df = data_con
data = [ dict(
        type = 'choropleth',
        locations = df['Code'],
        z = df['count'],
        text = df['Country'],
        colorscale = 'Viridis',
        autocolorscale = False,
        reversescale = True,
        marker = dict(
            line = dict (
                color = 'rgb(180,180,180)',
                width = 0.5
            ) ),
        colorbar = dict(
            autotick = False,
            tickprefix = '$',
            title = 'No of Crashes'),
      ) ]

layout = dict(
    title = 'Aviation Accidents by Country',
    geo = dict(
        showframe = False,
        showcoastlines = False,
        projection = dict(
            type = 'Mercator'
        )
    )
)

fig = dict( data=data, layout=layout )
py.iplot( fig, validate=False, filename='d3-world-map' )